In [13]:
import os
from PIL import Image
import base64
import json

In [14]:

def get_exif_data(image):
    """提取照片的EXIF数据"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]
                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value
    return exif_data

def get_coordinates(exif_data):
    """从EXIF数据中提取经纬度"""
    if "GPSInfo" in exif_data:      
        gps_info = exif_data["GPSInfo"]
        gps_latitude = gps_info.get("GPSLatitude")
        gps_latitude_ref = gps_info.get("GPSLatitudeRef")
        gps_longitude = gps_info.get("GPSLongitude")
        gps_longitude_ref = gps_info.get("GPSLongitudeRef")

        if gps_latitude and gps_longitude and gps_latitude_ref and gps_longitude_ref:
            lat = convert_to_degrees(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat = 0 - lat
            lon = convert_to_degrees(gps_longitude)
            if gps_longitude_ref != "E":
                lon = 0 - lon
            return lat, lon
    return None

# def convert_to_degrees(value):
#     """将度分秒格式转换为十进制度"""
#     d, m, s = value
#     return d + (m / 60.0) + (s / 360.00)
def generate_thumbnail(image_path, max_size=(100, 100)):
    """生成指定大小的缩略图"""
    with Image.open(image_path) as img:
        img.thumbnail(max_size)
        thumb_io = io.BytesIO()
        img.save(thumb_io, 'JPEG', quality=85)
        thumb_io.seek(0)
        thumb_base64 = base64.b64encode(thumb_io.getvalue()).decode('utf-8')
        return thumb_base64

def process_photos(folder_path):
    """ 处理文件夹中的照片 """
    photos = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            with Image.open(img_path) as img:
                exif_data = get_exif_data(img)
                coords = get_coordinates(exif_data)
                if coords:
                    thumbnail = generate_thumbnail(img_path)
                    photos.append({
                        'name': filename,
                        'lat': coords[0],
                        'lon': coords[1],
                        'thumbnail': f'data:image/jpeg;base64,{thumbnail}',
                        'image': img_path
                    })
    return photos

In [21]:
folder_path = r'D:\Pictures\2023\2023-12-15'
photos_data = process_photos(folder_path)
photos_json = json.dumps(photos_data)

html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Cesium Photos</title>
    <script src="https://cesium.com/downloads/cesiumjs/releases/1.82/Build/Cesium/Cesium.js"></script>
    <link href="https://cesium.com/downloads/cesiumjs/releases/1.82/Build/Cesium/Widgets/widgets.css" rel="stylesheet">
</head>
<body>
    <div id="cesiumContainer" style="width: 100%; height: 100vh;"></div>
    <script>
        var photoData = {photos_json};

        var viewer = new Cesium.Viewer('cesiumContainer');
        """  # Fix: Close the f-string expression here

html_template += """
        var entity = viewer.entities.add({
            position: Cesium.Cartesian3.fromDegrees(photo.lon, photo.lat),
            billboard: {
                image: photo.thumbnail,
                width: 50,
                height: 50
            }
        });

        entity.description = '<a href="' + photo.image + '" target="_blank"><img src="' + photo.image + '" style="width: 200px;"></a>';
    });
    </script>
</body>
</html>

"""

with open('photo_map.html', 'w') as file:
    file.write(html_template)


In [ ]:
folder_path = r'D:\Pictures\2023\2023-12-15'
photos_data = process_photos(folder_path)
photos_json = json.dumps(photos_data)

html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Cesium Photos</title>
    <script src="https://cesium.com/downloads/cesiumjs/releases/1.82/Build/Cesium/Cesium.js"></script>
    <link href="https://cesium.com/downloads/cesiumjs/releases/1.82/Build/Cesium/Widgets/widgets.css" rel="stylesheet">
</head>
<body>
    <div id="cesiumContainer" style="width: 100%; height: 100vh;"></div>
    <script>
        var photoData = {photos_json};

        var viewer = new Cesium.Viewer('cesiumContainer');

        photoData.forEach(function(photo) {
            var entity = viewer.entities.add({
                position: Cesium.Cartesian3.fromDegrees(photo.lon, photo.lat),
                billboard: {
                    image: photo.thumbnail,
                    width: 50,
                    height: 50
                }
            });

            entity.description = '<a href="' + photo.image + '" target="_blank"><img src="' + photo.image + '" style="width: 200px;"></a>';
        });
    </script>
</body>
</html>
"""

with open('photo_map.html', 'w') as file:
    file.write(html_template)
